In [10]:
# modules
import numpy as np
import pandas as pd
from pathlib import Path
import os
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, accuracy_score

In [11]:
# change dir
%cd /home/hirano/work/Competition/Nishika_cable/src/
%load_ext autoreload
%autoreload 2
!pwd

/home/hirano/work/Competition/Nishika_cable/src
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/hirano/work/Competition/Nishika_cable/src


In [15]:
os.makedirs('../output/ensemble', exist_ok=True)

pref = '15' #!!Please change Submit Number !!

# Do everything

In [42]:
import itertools
lis=['006', '007', '008', '009', '010', '030', '035', '037', '039', '043', '044', '045', '046']
result = []
for n in range(1,len(lis)+1):
    for conb in itertools.combinations(lis, n):
        result.append(list(conb)) #タプルをリスト型に変換
print(len(result))

8191


In [43]:
params = {
    'objective': 'multiclass',
    'num_class' : 15,
    'metrics': 'multi_error',
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'num_leaves': 32,
    'max_depth': 3,
    'learning_rate': 0.001,
    'reg_alpha': 0.5,
    'reg_lambda': 0,
}

## オリジナル

In [44]:
best = 0
best_scores = []

for ret in result:
    versions = ret
    
    if(3<=len(versions)<=5):
        for i, version in enumerate(versions):
            oof_name = [x for x in os.listdir(f'../output/results/') if f'oof_{version}' in x]

            if i==0:
                oof = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
                oof[f'pred_{i}'] = oof.pred.values
                oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
                     'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
                     'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

                sub = pd.read_csv(f'../output/results/sub_{version}.csv')
                sub[f'pred_{i}'] = sub.target.values
                sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
                     'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
                     'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

            else:
                _tmp = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
                oof[f'pred_{i}'] = _tmp.pred.values
                oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
                     'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
                     'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

                _tmp = pd.read_csv(f'../output/results/sub_{version}.csv')
                sub[f'pred_{i}'] = _tmp.target.values
                sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
                     'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
                     'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        oof = oof.reset_index(drop=True)

        stacking_oof = np.zeros(len(oof))
        #stacking_sub = np.zeros(len(sub))
        stacking_sub = []

        features = [f'pred_{i}' for i in range(len(versions))]
        features.extend(['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5',
                         'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11',
                         'conf_12', 'conf_13', 'conf_14'])
        #print(features)

        def lgb_f1_score(y_hat,data):
            y_true = data.get_label()
            y_hat = np.round(y_hat)
            return 'f1', f1_score(y_true, y_hat,average='macro'), True

        n_fold = 5

        for fold in range(n_fold):
            #print(f'{fold=}')

            trn = oof[oof.fold!=fold]
            val = oof[oof.fold==fold]

            val_idx = val.index

            trn_x = trn[features]
            trn_y = trn['target']
            val_x = val[features]
            val_y = val['target']

            tst_x = sub[features]

            model = LGBMClassifier(**params)
            model.fit(
                trn_x,
                trn_y,
                eval_metric='multi_logloss',
                eval_set=[(val_x, val_y)],
                verbose=-1,
                early_stopping_rounds=100
            )

            val_pred = model.predict(val_x)
            #stacking_sub += model.predict(tst_x)
            stacking_sub.append(model.predict(tst_x))

            stacking_oof[val_idx] = val_pred
            #break

        oof.pred = stacking_oof
        #print(oof.pred)

        score = f1_score(oof.target.values, oof.pred.values, average='micro')
        #print(f'Score is {score:.4f}')

        if score > best:
            best = score
            print('**********************************************')
            print(score, versions)
            print('**********************************************')
            best_scores.append([score, versions])
        #break

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[248]	valid_0's multi_logloss: 1.13237	valid_0's multi_error: 0.011194
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[232]	valid_0's multi_logloss: 1.1866	valid_0's multi_error: 0.0236318
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[239]	valid_0's multi_logloss: 1.15465	valid_0's multi_error: 0.00870647
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[241]	valid_0's multi_logloss: 1.16398	valid_0's multi_error: 0.0199005
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[264]	valid_0's multi_logloss: 1.11596	valid_0's multi_error: 0.0249066
**********************************************
0.9823339139089325 ['006', '007', '008']
**********************************************
Training until valida

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[271]	valid_0's multi_logloss: 1.0742	valid_0's multi_error: 0.0124378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[274]	valid_0's multi_logloss: 1.08346	valid_0's multi_error: 0.0211443
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[294]	valid_0's multi_logloss: 1.01708	valid_0's multi_error: 0.00746269
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[279]	valid_0's multi_logloss: 1.06685	valid_0's multi_error: 0.0136816
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[314]	valid_0's multi_logloss: 1.00188	valid_0's multi_error: 0.0199253
**********************************************
0.9850709131624782 ['006', '007', '045']
**********************************************
Training until valid

Early stopping, best iteration is:
[306]	valid_0's multi_logloss: 1.01643	valid_0's multi_error: 0.0174129
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[297]	valid_0's multi_logloss: 1.0414	valid_0's multi_error: 0.0298879
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[268]	valid_0's multi_logloss: 1.08222	valid_0's multi_error: 0.0124378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[256]	valid_0's multi_logloss: 1.12644	valid_0's multi_error: 0.0223881
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[234]	valid_0's multi_logloss: 1.16876	valid_0's multi_error: 0.011194
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[333]	valid_0's multi_logloss: 0.957925	valid_0's multi_error: 0.0174129
Training until validation scores don't improve fo

Early stopping, best iteration is:
[265]	valid_0's multi_logloss: 1.09042	valid_0's multi_error: 0.00870647
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[255]	valid_0's multi_logloss: 1.12714	valid_0's multi_error: 0.0161692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[447]	valid_0's multi_logloss: 0.786781	valid_0's multi_error: 0.0249066
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[282]	valid_0's multi_logloss: 1.05147	valid_0's multi_error: 0.0124378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[285]	valid_0's multi_logloss: 1.0645	valid_0's multi_error: 0.0199005
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[250]	valid_0's multi_logloss: 1.13113	valid_0's multi_error: 0.0136816
Training until validation scores don't improve 

Early stopping, best iteration is:
[238]	valid_0's multi_logloss: 1.17336	valid_0's multi_error: 0.0211443
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[222]	valid_0's multi_logloss: 1.2	valid_0's multi_error: 0.00746269
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[306]	valid_0's multi_logloss: 1.01003	valid_0's multi_error: 0.0161692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[234]	valid_0's multi_logloss: 1.18805	valid_0's multi_error: 0.0236613
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[263]	valid_0's multi_logloss: 1.09396	valid_0's multi_error: 0.00995025
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[272]	valid_0's multi_logloss: 1.09247	valid_0's multi_error: 0.0223881
Training until validation scores don't improve for

...

Early stopping, best iteration is:
[286]	valid_0's multi_logloss: 1.05868	valid_0's multi_error: 0.0199253
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[342]	valid_0's multi_logloss: 0.925763	valid_0's multi_error: 0.0124378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[264]	valid_0's multi_logloss: 1.10778	valid_0's multi_error: 0.0199005
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[206]	valid_0's multi_logloss: 1.24455	valid_0's multi_error: 0.00995025
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[299]	valid_0's multi_logloss: 1.02716	valid_0's multi_error: 0.0161692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[300]	valid_0's multi_logloss: 1.02813	valid_0's multi_error: 0.0211706
Training until validation scores don't improve

Early stopping, best iteration is:
[265]	valid_0's multi_logloss: 1.10625	valid_0's multi_error: 0.0186567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[232]	valid_0's multi_logloss: 1.18985	valid_0's multi_error: 0.0236613
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[297]	valid_0's multi_logloss: 1.01676	valid_0's multi_error: 0.011194
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[238]	valid_0's multi_logloss: 1.1683	valid_0's multi_error: 0.0223881
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[214]	valid_0's multi_logloss: 1.22123	valid_0's multi_error: 0.011194
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[316]	valid_0's multi_logloss: 0.991142	valid_0's multi_error: 0.0149254
Training until validation scores don't improve for

Early stopping, best iteration is:
[252]	valid_0's multi_logloss: 1.12247	valid_0's multi_error: 0.0124378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[325]	valid_0's multi_logloss: 0.973634	valid_0's multi_error: 0.0161692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[236]	valid_0's multi_logloss: 1.17738	valid_0's multi_error: 0.0199253
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[363]	valid_0's multi_logloss: 0.885738	valid_0's multi_error: 0.00995025
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[286]	valid_0's multi_logloss: 1.05339	valid_0's multi_error: 0.0186567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[211]	valid_0's multi_logloss: 1.22919	valid_0's multi_error: 0.00995025
Training until validation scores don't impro

Early stopping, best iteration is:
[227]	valid_0's multi_logloss: 1.19876	valid_0's multi_error: 0.0186567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[259]	valid_0's multi_logloss: 1.10007	valid_0's multi_error: 0.00746269
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[268]	valid_0's multi_logloss: 1.10225	valid_0's multi_error: 0.0161692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[287]	valid_0's multi_logloss: 1.05602	valid_0's multi_error: 0.0174346
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[339]	valid_0's multi_logloss: 0.929669	valid_0's multi_error: 0.0136816
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[271]	valid_0's multi_logloss: 1.0849	valid_0's multi_error: 0.0186567
Training until validation scores don't improve 

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[340]	valid_0's multi_logloss: 0.928642	valid_0's multi_error: 0.011194
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[254]	valid_0's multi_logloss: 1.12684	valid_0's multi_error: 0.0186567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[220]	valid_0's multi_logloss: 1.20435	valid_0's multi_error: 0.00870647
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[300]	valid_0's multi_logloss: 1.02736	valid_0's multi_error: 0.0149254
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[281]	valid_0's multi_logloss: 1.07178	valid_0's multi_error: 0.0211706
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[292]	valid_0's multi_logloss: 1.02838	valid_0's multi_er

In [45]:
best_scores

[[0.9823339139089325, ['006', '007', '008']],
 [0.9833291863647674, ['006', '007', '009']],
 [0.983578004478726, ['006', '007', '010']],
 [0.9840756407066434, ['006', '007', '030']],
 [0.9848220950485196, ['006', '007', '035']],
 [0.9850709131624782, ['006', '007', '045']],
 [0.9853197312764369, ['006', '030', '037']],
 [0.9858173675043543, ['007', '010', '037']],
 [0.986066185618313, ['007', '035', '044']],
 [0.9863150037322717, ['007', '037', '044']],
 [0.9865638218462304, ['007', '039', '043', '045']],
 [0.9868126399601891, ['035', '039', '044', '045']]]

In [6]:
best_scores

[[0.9865638218462304, ['010']],
 [0.9870614580741478, ['037']],
 [0.9878079124160238, ['010', '035']],
 [0.9890520029858174, ['010', '037']],
 [0.989300821099776, ['010', '044', '045', '046']]]

0.989300821099776 ['010', '044', '045', '046']

## 横方向に拡張

# One version

In [44]:
# Load
for i, version in enumerate(versions):
    oof_name = [x for x in os.listdir(f'../output/results/') if f'oof_{version}' in x]
    
    if i==0:
        oof = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'pred_{i}'] = oof.pred.values
        oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        sub = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'pred_{i}'] = sub.target.values
        sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

    else:
        _tmp = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'pred_{i}'] = _tmp.pred.values
        oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        _tmp = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'pred_{i}'] = _tmp.target.values
        sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

oof = oof.reset_index(drop=True)

['oof_010_0.9868.csv']
['oof_035_0.9861.csv']
['oof_037_0.9881.csv']
['oof_039_0.9846.csv']
['oof_043_0.9816.csv']
['oof_044_0.9876.csv']
['oof_045_0.9863.csv']
['oof_046_0.8184.csv']


In [45]:
oof.head()

,filename,Xmin_Ymin_Xmax_Ymax,class,filename_trim,fold,target,pred,conf_0,conf_1,conf_2,...,conf_13,conf_14,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,00037f39.jpg,NaN,Lightning,00037f39_trim.jpg,1,3,3,0.003787,0.002242,0.000460,...,0.000523,0.000694,3,3,3,3,3,3,3,3
1,000c15d0.jpg,"[906, 1686, 1294, 1946]",USB_Type_C,NaN,2,8,8,0.000381,0.000753,0.000645,...,0.000197,0.000204,8,8,8,8,8,8,8,8
2,000f21ac.jpg,NaN,USB_Type_C,000f21ac_trim.jpg,3,8,8,0.000515,0.000813,0.001357,...,0.000584,0.000899,8,8,8,8,8,8,8,8
3,0013fa17.jpg,"[1458, 1756, 2304, 2602]",DisplayPort,NaN,0,14,14,0.004671,0.004009,0.005491,...,0.005857,2.391453,14,14,14,14,14,14,14,14
4,00150bc1.jpg,"[1181, 1692, 1658, 1985]",USB_Type_C,NaN,2,8,8,0.000951,0.000856,0.000881,...,0.000319,0.001991,8,8,8,8,8,8,8,8


In [46]:
sub.head()

,target,conf_0,conf_1,conf_2,conf_3,conf_4,conf_5,conf_6,conf_7,conf_8,...,conf_13,conf_14,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,8,0.002596,0.060591,0.001651,0.001756,0.033146,0.003705,0.001423,0.002175,2.520343,...,0.002329,0.001181,8,8,8,8,8,8,8,8
1,4,0.000754,0.018394,0.000324,0.004225,2.599620,0.002920,0.000808,0.003255,0.003617,...,0.000483,0.000752,4,4,4,4,4,4,4,4
2,8,0.000672,0.017930,0.000321,0.005417,0.254959,0.000479,0.000649,0.002937,2.350251,...,0.000353,0.000362,8,8,8,8,8,8,8,4
3,4,0.001937,0.004072,0.000569,0.010337,2.254389,0.002191,0.000684,0.002853,0.328717,...,0.000800,0.000814,4,4,4,4,4,4,4,8
4,1,0.001623,2.389181,0.000270,0.005252,0.193691,0.004907,0.000664,0.002687,0.031186,...,0.000316,0.000342,1,1,1,1,1,1,1,4


In [47]:
params = {
    'objective': 'multiclass',
    'num_class' : 15,
    'metrics': 'multi_error',
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'num_leaves': 32,
    'max_depth': 2,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.3,
    'bagging_freq': 5,
    'reg_alpha': 0.5,
    'reg_lambda': 0,
}

In [54]:
stacking_oof = np.zeros(len(oof))
#stacking_sub = np.zeros(len(sub))
stacking_sub = []

features = [f'pred_{i}' for i in range(len(versions))]
features.extend(['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5',
                 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11',
                 'conf_12', 'conf_13', 'conf_14'])
print(features)

def lgb_f1_score(y_hat,data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1', f1_score(y_true, y_hat,average='macro'), True

n_fold = 5

for fold in range(n_fold):
    print(f'{fold=}')

    trn = oof[oof.fold!=fold]
    val = oof[oof.fold==fold]

    val_idx = val.index

    trn_x = trn[features]
    trn_y = trn['target']
    val_x = val[features]
    val_y = val['target']

    tst_x = sub[features]

    model = LGBMClassifier(**params)
    model.fit(
        trn_x,
        trn_y,
        eval_metric='multi_logloss',
        eval_set=[(val_x, val_y)],
        verbose=100,
        early_stopping_rounds=300
    )
    
    val_pred = model.predict(val_x)
    #stacking_sub += model.predict(tst_x)
    stacking_sub.append(model.predict(tst_x))
    
    stacking_oof[val_idx] = val_pred
    #

    oof.pred = stacking_oof
    #print(oof.pred)

    score = f1_score(oof.target.values, oof.pred.values, average='micro')
    #print(f'Score is {score:.4f}')

['pred_0', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4', 'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9', 'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']
fold=0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 300 rounds
[100]	valid_0's multi_logloss: 0.280039	valid_0's multi_error: 0.00746269
[200]	valid_0's multi_logloss: 0.0798369	valid_0's multi_error: 0.00746269
[300]	valid_0's multi_logloss: 0.0479741	valid_0's multi_error: 0.00746269
Early stopping, best iteration is:
[26]	valid_0's multi_logloss: 1.08959	valid_0's multi_error: 0.00746269
fold=1
[LightGB

# OOF

In [55]:
oof.pred = stacking_oof
print(oof.pred)

score = f1_score(oof.target.values, oof.pred.values, average='micro')
print(f'Score is {score:.4f}')

#sub_name = '_'.join(versions)
#oof.to_csv(f'../output/ensemble/{pref}_oof_stacking_{sub_name}_{score:.4f}.csv', index=False)

0        3.0
1        8.0
2        8.0
3       14.0
4        8.0
        ... 
4014     4.0
4015     3.0
4016     6.0
4017     3.0
4018     4.0
Name: pred, Length: 4019, dtype: float64
Score is 0.9876


# Submission

In [50]:
label_dict = {
   0 : 'HDMI',
   1 : 'USB_Mini',
   2 : 'Dock',
   3 : 'Lightning',
   4 : 'USB_Micro_B',
   5 : 'Mini_DisplayPort',
   6 : 'RJ_45',
   7 : 'USB_Type_A',
   8 : 'USB_Type_C',
   9 : 'USB_Micro_B_3.1',
  10 : 'USB_Type_B',
  11 : 'Lightning_T',
  12 : 'USB_Micro_B_W',
  13 : 'VGA',
  14 : 'DisplayPort'
}

In [51]:
import scipy.stats as stats

#print(stacking_sub)
results = stats.mode(stacking_sub, axis=0).mode[0]
print(len(results))

# class num to class name
targets = []
for i in range(len(results)):
    targets.append(label_dict[results[i]])
sub['class'] = targets
print(len(targets))

test = pd.read_csv('../input/test_with_fold.csv')
sub['filename'] = test['filename']

sub_name = '_'.join(versions)
sub[['filename', 'class']].to_csv(f'../output/ensemble/{pref}_stacking_{sub_name}.csv', index=False)

2200
2200
